In [ ]:
!pip install dnspython
!pip install pymongo[srv]

In [ ]:
import pymongo
client = pymongo.MongoClient("mongodb+srv://<username>:<password>@cluster0.kjtaz.mongodb.net/myFirstDatabase?retryWrites=true&w=majority")
db = client.studentsdb
rec=db.studentscol

In [ ]:
import json
with open("students.json","r") as fyl:
  a=[]
  for i in fyl:
    a.append(json.loads(i))

In [ ]:
rec.insert_many(a)

In [ ]:
b=rec.aggregate(
   [{"$unwind":"$scores"},
     {
       "$group":
         {"_id":"$scores.type",
          "maxmark":{"$max":"$scores.score"},
         }
     }
   ]
)
for i in b:
  c=i["_id"]
  d=i["maxmark"]
  for j in rec.find({"scores.type":c,"scores.score":d}):
    print(j)

In [ ]:
e=rec.aggregate(
   [{"$unwind":"$scores"},
     {
       "$group":
         {"_id":"$_id",
          "total":{"$sum":"$scores.score"},
         }
     }
   ]
)
f=0
g=""
for i in e:
  if i["total"]>f:
    f=i["total"]
    g=i["_id"]
for i in rec.find({"_id":g},{"name":1}):
  print(i)

In [ ]:
h=rec.aggregate(
   [{"$unwind":"$scores"},{"$match":{"scores.type":"exam"}},
     {
       "$group":
         {"_id":"$scores.type",
           "avgexam": { "$avg": "$scores.score" }
         }
     }
   ]
)
for i in h:
  avg_exam=i["avgexam"]
for i in rec.find({"scores.0.score":{"$gt":40,"$lt":avg_exam}}):
  print(i)

In [ ]:
rec.update_many({"$or":[{"scores.0.score":{"$lt":40}},{"scores.1.score":{"$lt":40}},{"scores.2.score":{"$lt":40}}]},{"$set":{"results":"fail"}})

In [ ]:
rec.update_many({"scores.0.score":{"$gt":40},"scores.1.score":{"$gt":40},"scores.2.score":{"$gt":40}},{"$set":{"results":"pass"}})

In [ ]:
tarec=db.totalaveragecol

In [ ]:
ab=rec.aggregate(
   [{"$unwind":"$scores"},
     {
       "$group":
         {"_id":"$scores.type",
          "total":{"$sum":"$scores.score"},
           "average": { "$avg": "$scores.score" }
         }
     }
   ]
)

In [ ]:
tarec.insert_many(ab)

In [ ]:
tasrec=db.totalaveragestudentscol

In [ ]:
cd=rec.aggregate(
   [{"$unwind":"$scores"},
     {
       "$group":
         {"_id":"$_id",
          "total":{"$sum":"$scores.score"},
           "average": { "$avg": "$scores.score" }
         }
     }
   ]
)

In [ ]:
tasrec.insert_many(cd)

In [ ]:
newrec=db.newcol

In [ ]:
a1=rec.aggregate(
   [{"$unwind":"$scores"},{"$match":{"scores.type":"exam"}},
     {
       "$group":
         {"_id":"$scores.type",
           "avgexam": { "$avg": "$scores.score" }
         }
     }
   ]
)
for i in a1:
  avg_exam=i["avgexam"]
b1=rec.aggregate(
   [{"$unwind":"$scores"},{"$match":{"scores.type":"quiz"}},
     {
       "$group":
         {"_id":"$scores.type",
           "avgquiz": { "$avg": "$scores.score" }
         }
     }
   ]
)
for i in b1:
  avg_quiz=i["avgquiz"]
c1=rec.aggregate(
   [{"$unwind":"$scores"},{"$match":{"scores.type":"homework"}},
     {
       "$group":
         {"_id":"$scores.type",
           "avghomework": { "$avg": "$scores.score" }
         }
     }
   ]
)
for i in c1:
  avg_homework=i["avghomework"]
abc=rec.find({"scores.0.score":{"$gt":40,"$lt":avg_exam},"scores.1.score":{"$gt":40,"$lt":avg_quiz},"scores.2.score":{"$gt":40,"$lt":avg_homework}})

In [ ]:
newrec.insert_many(abc)

In [ ]:
failrec=db.failedsudentscol

In [ ]:
ef=rec.find({"results":"fail"})

In [ ]:
failrec.insert_many(ef)

In [ ]:
passrec=db.passedstudentscol

In [ ]:
gh=rec.find({"results":"pass"})

In [ ]:
passrec.insert_many(gh)